In [ ]:
# For google colab
#!pip install erddap-python

In [22]:
import matplotlib.pyplot as plt
from erddapClient import ERDDAP_Griddap
import pandas as pd
from pathlib import Path
import imageio
from tqdm import tqdm
import xarray as xr
import warnings
from loguru import logger

import matplotlib
matplotlib.use('Agg')

warnings.simplefilter(action="ignore", category=FutureWarning)

In [23]:
@logger.catch
def get_data(server, datasetID, vars, time, nc_output_file):
    subset = (
        ERDDAP_Griddap(server, datasetID)
        .setResultVariables(list(vars.keys()))
        .setSubset(
            time=time,
            depth=slice(0.5000003, 0.5000003),
            gridY=slice(600, 897),
            gridX=slice(100, 300),
        )
        .getxArray()
    )
    # save dataset
    # Drop some atttributes that fails to be encoded
    for var in subset.variables:
        subset[var].attrs.pop("_evenlySpaced", None)
        subset[var].attrs.pop("actual_range", None)
        subset[var].attrs.pop("units", None)
        subset[var].attrs.pop("calendar", None)
    subset.to_netcdf(nc_output_file)
    return subset

In [24]:
server = "https://salishsea.eos.ubc.ca/erddap"
datasetID = "ubcSSg3DPhysicsFields1hV21-11"
vars = {
    "temperature": {"vmin": 5, "vmax": 25, "cmap": "inferno"},
    "salinity": {"vmin": 20, "vmax": 32, "cmap": "viridis"},
}
get_data(server, datasetID, vars,"2021-01-01T00:30:00Z", "test.nc")

<xarray.Dataset>
Dimensions:      (time: 1, depth: 1, gridY: 298, gridX: 201)
Coordinates:
  * time         (time) object 2021-01-01 00:30:00
  * depth        (depth) float64 0.5
  * gridY        (gridY) float64 600.0 601.0 602.0 603.0 ... 895.0 896.0 897.0
  * gridX        (gridX) float64 100.0 101.0 102.0 103.0 ... 298.0 299.0 300.0
Data variables:
    temperature  (time, depth, gridY, gridX) float32 ...
    salinity     (time, depth, gridY, gridX) float32 ...
Attributes: (12/25)
    acknowledgement:           MEOPAR, Ocean Networks Canada (ONC), Digital R...
    cdm_data_type:             Grid
    comment:                   If you use this dataset in your research,\nple...
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    creator_email:             sallen@eoas.ubc.ca
    creator_name:              SalishSeaCast Project Contributors
    ...                        ...
    testOutOfDate:             now-16hours
    time_coverage_end:         2024-02-08T23:30:00Z
    time_coverage_start:       2007-01-01T00:30:00Z
    timeStamp:                 2024-Feb-08 17:19:42 GMT
    title:                     Green, Salish Sea, 3d Physics Fields, Hourly, ...
    uuid:                      71c2f2a3-a48b-4056-9bd3-c4c786f94607

In [25]:
@logger.catch
def get_frame(time, vars, output, server, datasetID):
    """Get water properties frames"""
    nc_output_file = Path(output.format(time=time, var="-".join(vars.keys())) + ".nc")
    if nc_output_file.exists():
        subset = xr.open_dataset(nc_output_file)
    else:
        subset = get_data(server, datasetID, vars, time, nc_output_file)

    for var, attrs in vars.items():
        _, ax = plt.subplots()
        subset[var].plot(ax=ax, **attrs)
        plt.savefig(output.format(time=time, var=var))

## Time Range of Interest


In [26]:
time_frames = pd.date_range(
    start=pd.to_datetime("2016-07-25T00:30:00Z"),
    end=pd.to_datetime("2016-08-05T00:30:00Z"),
    freq=pd.Timedelta("1h"),
)
Path("output").mkdir(parents=True, exist_ok=True)

## Water Properties 

In [27]:
# Get temperature,salinity frames
server = "https://salishsea.eos.ubc.ca/erddap"
datasetID = "ubcSSg3DPhysicsFields1hV21-11"

vars = {
    "temperature": {"vmin": 5, "vmax": 25, "cmap": "inferno"},
    "salinity": {"vmin": 20, "vmax": 32, "cmap": "viridis"},
}

data = []
for time in tqdm(time_frames, desc="Generate frames", unit="time_stamp"):
    get_frame(
        time.isoformat().replace("+00:00", "Z"),
        vars,
        "output/{var}_{time}",
        server,
        datasetID,
    )

Generate frames:   4%|▍         | 10/265 [00:02<01:01,  4.14time_stamp/s]/var/folders/bv/k82zg3_x1fg00tjywdqys88h0000gn/T/ipykernel_77625/2377147092.py:11: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  _, ax = plt.subplots()
Generate frames: 100%|██████████| 265/265 [01:00<00:00,  4.41time_stamp/s]


## Load Current data

In [36]:
subset = (
    ERDDAP_Griddap("https://salishsea.eos.ubc.ca/erddap", "ubcSSg3DuGridFields1hV21-11")
    .setResultVariables("uVelocity")
    .setSubset(
        time=time,
        depth=slice(0.5000003, 0.5000003),
        gridY=slice(600, 897),
        gridX=slice(100, 300),
    )
    .getxArray()
)

Note: Caching=1


In [37]:
server = "https://salishsea.eos.ubc.ca/erddap"
datasetIDs = {"uVelocity": "ubcSSg3DuGridFields1hV21-11", "vVelocity": "ubcSSg3DvGridFields1hV21-11"}

for time in tqdm(time_frames, desc="Generate frames", unit="time_stamp"):
    for var, datasetID in datasetIDs.items():
        get_frame(
            time.isoformat().replace("+00:00", "Z"),
            {var: {"vmin": -1, "vmax": 1, "cmap": "coolwarm"}},
            "output/{var}_{time}",
            server,
            datasetID,
        )

Generate frames:   0%|          | 0/265 [00:00<?, ?time_stamp/s]Note: Caching=1
Note: Caching=1
Generate frames:   0%|          | 1/265 [00:08<38:14,  8.69s/time_stamp]Note: Caching=1
Note: Caching=1
Generate frames:   1%|          | 2/265 [00:15<33:53,  7.73s/time_stamp]Note: Caching=1
Note: Caching=1
Generate frames:   1%|          | 3/265 [00:24<36:46,  8.42s/time_stamp]Note: Caching=1
Note: Caching=1
Generate frames:   2%|▏         | 4/265 [00:32<34:26,  7.92s/time_stamp]Note: Caching=1
Note: Caching=1
Generate frames:   2%|▏         | 5/265 [00:39<33:08,  7.65s/time_stamp]Note: Caching=1
Note: Caching=1
Generate frames:   2%|▏         | 6/265 [00:46<32:19,  7.49s/time_stamp]Note: Caching=1
Note: Caching=1
Generate frames:   3%|▎         | 7/265 [00:55<34:45,  8.09s/time_stamp]Note: Caching=1
Note: Caching=1
Generate frames:   3%|▎         | 8/265 [01:03<33:29,  7.82s/time_stamp]Note: Caching=1
Note: Caching=1
Generate frames:   3%|▎         | 9/265 [01:10<32:26,  7.60s/time_stamp]

## Load 

## Load Sentry Shoal

In [20]:
## Load Sentry Shoal data

In [ ]:
# Generate animation gif
frames = []
for var in ["temperature", "salinity"]:
    for frame in sorted(Path("output").glob(f"{var}_*.png")):
        image = imageio.v2.imread(frame)
        frames.append(image)
        
    imageio.mimsave(
        f"{var}_animation.gif", frames, fps=4  # output gif  # array of input frames
    )